# Cricket T20 World Cup Data Analytics

In [1]:
# importing the libraries
import pandas as pd
import json

Now we will load the data from the JSON file and create a dataframe out of it

<h3 style="color:Red"> 1. Match Results File </h3>

In [10]:
with open('Data/json_files/t20_wc_match_results.json') as file:
    result_data =json.load(file)

result_df = pd.DataFrame(result_data[0]['matchSummary'])
result_df.head()


,team1,team2,winner,margin,ground,matchDate,scorecard
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


In [13]:
# We will use the scorecard column as a unique key to indentify the different matches and will also be used to link this 
# dataframe with other like a primary key

result_df.rename({'scorecard':'match_id'}, axis = 1, inplace = True)
result_df.head()

,team1,team2,winner,margin,ground,matchDate,match_id
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


Mapping team names to a unique match_id. This will be used to link with other tables

In [60]:
# Create dictionary that holds the data as follows {team1 Vs team2 : match_id, team2 Vs team1 : match_id, etcc... for each row}

# Function iterrows() -> It will iterate over the rows one after another in a data frame

match_details = {}

for index,row in result_df.iterrows():
    key1 = row['team1'] + ' Vs ' + row['team2']
    key2 = row['team2'] + ' Vs ' + row['team1']
    
    match_details[key1] = row['match_id']
    match_details[key2] = row['match_id']
    
#match_details.keys()

In [89]:
#Exporting data file to csv file
result_df.to_csv("Data/match_summary.csv", index = False)

<h3 style="color:Red">2. Batting Summary <h3>

In [33]:
with open('Data/json_files/t20_wc_batting_summary.json') as file:
    batting_Summary = json.load(file)
    
    
# As we see that data is not present in a singal list, it has battingSummary item for each match hence making it multidimensional
# we will now convert the multidimensional list into a single dimension.
    batting_data = []
    for record in batting_Summary:
        batting_data.extend(record['battingSummary'])

print("Compare lengths of batting_summary vs batting data {} vs {}".format(len(batting_Summary),len(batting_data)))       



Compare lengths of batting_summary vs batting data 45 vs 699


In [38]:
#Create a dataframe from the above 1D data
batting_df = pd.DataFrame(batting_data)
batting_df.head()

,match,teamInnings,battingPos,batsmanName,dismissal,runs,balls,4s,6s,SR
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,c Pramod Madushan b Chameera,3,6,0,0,50.00
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,c Shanaka b Pramod Madushan,9,9,1,0,100.00
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,c â€ Mendis b Karunaratne,20,12,1,2,166.66
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,c DM de Silva b Pramod Madushan,26,24,2,0,108.33
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),c Gunathilaka b PWH de Silva,20,24,0,0,83.33


In [41]:
# Based on dismissal column we will create a new column to identify if the player is out or not, once created we can drop the
# dismissal column from the dataframe
batting_df["out/notout"] = batting_df.dismissal.apply(lambda x: "notout" if len(x) == 0 else "out" )
batting_df.head(10)

,match,teamInnings,battingPos,batsmanName,dismissal,runs,balls,4s,6s,SR,out/notout
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,c Pramod Madushan b Chameera,3,6,0,0,50.00,out
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,c Shanaka b Pramod Madushan,9,9,1,0,100.00,out
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,c â€ Mendis b Karunaratne,20,12,1,2,166.66,out
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,c DM de Silva b Pramod Madushan,26,24,2,0,108.33,out
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),c Gunathilaka b PWH de Silva,20,24,0,0,83.33,out
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,run out (Gunathilaka/â€ Mendis),44,28,4,0,157.14,out
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,c â€ Mendis b Theekshana,0,1,0,0,0.00,out
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,,31,16,2,2,193.75,notout
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,c Smit b Shikongo,9,10,1,0,90.00,out
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendisâ€,c â€ Green b Wiese,6,6,0,0,100.00,out


In [43]:
# Dropping the dismissal column as it is no longer required
batting_df.drop(columns = ["dismissal"], inplace = True)
batting_df.head()

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/notout
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,out
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,out
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,out
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,out
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,out


In [47]:
#Removing the bad characters
batting_df['batsmanName'] = batting_df['batsmanName'].apply(lambda x : x.replace('â€',''))
batting_df['batsmanName'] = batting_df['batsmanName'].apply(lambda x : x.replace('\xa0',''))

batting_df['batsmanName'][9]

'Kusal Mendis'

Creating and formatting criteria to join both the data frames

In [57]:
# Using the dictionary to add a new column match_id into the data frame:

batting_df['match_id'] = batting_df['match'].map(match_details)

batting_df.head()
#match_details['Namibia Vs Sri Lanka']

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/notout,match_id
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,out,T20I # 1823
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,out,T20I # 1823
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,out,T20I # 1823
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,out,T20I # 1823
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,out,T20I # 1823


In [59]:
# Exporting the table to a csv file
batting_df.to_csv("Data/batting_summary.csv", index = False)

<h3 style="color:Red">3. Bowling Summary </h3>

In [70]:
with open("Data/json_files/t20_wc_bowling_summary.json") as file:
    bowling_data = json.load(file)
# Same as we did for batting summary file, we need to convet this 2_D list to 1 dimension (Dimension Reduction)
    bowlingSummary = []
    for record in bowling_data:
        bowlingSummary.extend(record['bowlingSummary'])

bowlingSummary[:5] # Top 5 elements 

In [77]:
bowling_df = pd.DataFrame(bowlingSummary)
print(bowling_df.shape)
bowling_df.head(4)

(500, 13)


,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0


In [78]:
# Map the dataframe so that it contains the match_id
bowling_df['match_id'] = bowling_df['match'].map(match_details)

bowling_df.shape

(500, 14)

In [79]:
bowling_df.to_csv("Data/bowling_summary.csv", index = False)

<h3 style="color:Red">4. Player Information </h3>

In [81]:
with open("Data/json_files/t20_wc_player_info.json") as file:
    player_info = json.load(file)

player_info[1]

{'name': 'Divan la Cock',
 'team': 'Namibia',
 'battingStyle': 'Right hand Bat',
 'bowlingStyle': 'Legbreak',
 'playingRole': 'Opening Batter',
 'description': ''}

In [82]:
player_df = pd.DataFrame(player_info)

In [84]:
player_df.shape

(219, 6)

In [85]:
player_df.head()

,name,team,battingStyle,bowlingStyle,playingRole,description
0,Michael van Lingen,Namibia,Left hand Bat,Left arm Medium,Bowling Allrounder,
1,Divan la Cock,Namibia,Right hand Bat,Legbreak,Opening Batter,
2,Jan Nicol Loftie-Eaton,Namibia,Left hand Bat,"Right arm Medium, Legbreak",Batter,
3,Stephan Baard,Namibia,Right hand Bat,Right arm Medium fast,Batter,
4,Gerhard Erasmus(c),Namibia,Right hand Bat,Right arm Offbreak,Allrounder,


In [86]:
# Cleaning data by removing invalid characters.

player_df["name"] = player_df["name"].apply(lambda x: x.replace('â€', ''))
player_df["name"] = player_df["name"].apply(lambda x: x.replace('\xa0', ''))
player_df["name"] = player_df["name"].apply(lambda x: x.replace('†', ''))

player_df["name"].head(10)

0        Michael van Lingen
1             Divan la Cock
2    Jan Nicol Loftie-Eaton
3             Stephan Baard
4        Gerhard Erasmus(c)
5              Jan Frylinck
6               David Wiese
7                   JJ Smit
8           Pathum Nissanka
9              Kusal Mendis
Name: name, dtype: object

In [87]:
# Indian Players

player_df[player_df['team'] == 'India']

,name,team,battingStyle,bowlingStyle,playingRole,description
127,KL Rahul,India,Right hand Bat,,Opening Batter,"A tall, elegant right-hand batsman who can kee..."
128,Rohit Sharma(c),India,Right hand Bat,Right arm Offbreak,Top order Batter,"Languid and easy on the eye, Rohit Sharma owne..."
129,Virat Kohli,India,Right hand Bat,Right arm Medium,Top order Batter,India has given to the world many a great cric...
130,Suryakumar Yadav,India,Right hand Bat,"Right arm Medium, Right arm Offbreak",Batter,Hard-hitting 360-degree batter Suryakumar Yada...
131,Axar Patel,India,Left hand Bat,Slow Left arm Orthodox,Bowling Allrounder,Left-arm spinner Axar Patel has been increasin...
132,Hardik Pandya,India,Right hand Bat,Right arm Medium fast,Allrounder,Hardik Pandya swears by living life king size ...
133,Dinesh Karthik,India,Right hand Bat,Right arm Offbreak,Wicketkeeper Batter,Not many would forget the sight of Dinesh Kart...
134,Ravichandran Ashwin,India,Right hand Bat,Right arm Offbreak,Bowling Allrounder,R Ashwin took the tricks and skills he learned...
135,Bhuvneshwar Kumar,India,Right hand Bat,Right arm Medium,Bowler,"At the time of his India debut in 2012, Bhuvne..."
136,Arshdeep Singh,India,Left hand Bat,Left arm Medium fast,Bowler,


In [88]:
# Save file as CSV

player_df.to_csv("Data/player_summary.csv", index = False)